PROJECT 1 — LOAN DEFAULT RISK (LENDING CLUB)

Objective:
Build a credit risk model to predict loan default and support approval decisions, risk segmentation, and portfolio risk management.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_csv("data/raw/lending_club_loan_two.csv")

df.head()



NameError: name 'pd' is not defined